In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,confusion_matrix
from sklearn.tree import export_graphviz
import graphviz
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scipy.stats import shapiro
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor

from statsmodels.api import Logit
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import chi2_contingency

import warnings
warnings.filterwarnings("ignore")

# path = "C:/Users/cokr3/OneDrive/바탕 화면/이차전지 프로젝트/"

def eval_class_model(y_test, y_pred):
    confusion = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test,y_pred)
    recall = f1_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    AUC = roc_auc_score(y_test,y_pred)
    print('혼돈행렬 : \n{}'.format(confusion))
    print('accuracy : {:.4f}'.format(accuracy))
    print('precision : {:.4f}'.format(precision))
    print('recall : {:.4f}'.format(recall))
    print('f1 : {:.4f}'.format(f1))
    print('AUC : {:.4f}'.format(AUC))

#%% RAW data
df_raw_tat = pd.read_csv("bat_tat.csv")
df_raw_process = pd.read_csv("bat_process.csv", encoding = 'cp949')

#%% 데이터 전처리 ----------------
#%% Drop_duplicated

"""
lot_id, tray_id의 중복 값이 존재하나, cell_id 단위로 보았을 때 데이터가 달랐던 점을 생각하여,
해당 부분에 대해선 중복제거를 진행하지 않는다
"""

#%% DropNa
"""
tat : 결측치 없음
process : 결측치 존재 -> 행삭제
"""

df_process_na = df_raw_process[df_raw_process.isna().any(axis=1)]
df_process = df_raw_process.drop(df_process_na[ df_process_na["judge"] == "양품"].index, axis = 0).reset_index(drop = True)

#%% 이상치 제거

df_tat = df_raw_tat.copy()

#1. rta2_tat
print("rta2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat= df_tat[ df_tat["rta2_tat"] >= 6000 ].reset_index(drop = True)
print("rta2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#2. ocv1_tat
print("ocv1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["ocv1_tat"] <= 140 ].reset_index(drop = True)
print("ocv1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#3. c1_tat
print("c1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c1_tat"] <= 6000 ].reset_index(drop = True)
print("c1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#4. c2_tat
print("c2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c2_tat"] <= 1000 ].reset_index(drop = True)
print("c2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#5. dc2_tat
print("dc2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["dc2_tat"] <= 460 ].reset_index(drop = True)
print("dc2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#6. c3_tat
print("c3_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c3_tat"] <= 10000 ].reset_index(drop = True)
print("c3_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#7. dc3_tat
print("dc3_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["dc3_tat"] <= 7000 ].reset_index(drop = True)
print("dc3_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#8. c4_tat
print("c4_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c4_tat"] <= 14000 ].reset_index(drop = True)
print("c4_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#9. pg1_tat
print("pg1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["pg1_tat"] <= 800 ].reset_index(drop = True)
df_tat = df_tat[ df_tat["pg1_tat"] >= 200 ].reset_index(drop = True)
print("pg1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#10. sa1_tat
print("sa1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["sa1_tat"] >= 2000 ].reset_index(drop = True)
print("sa1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#11. sa2_tat
print("sa2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["sa2_tat"] >= 250000 ].reset_index(drop = True)
print("sa2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#12. sa3_tat
print("sa3_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["sa3_tat"] >= 190000 ].reset_index(drop = True)
print("sa3_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')


"""
01. rta2_tat 이상치 제거 : 61개
02. ocv1_tat 이상치 제거 : 1개
03. c1_tat 이상치 제거 : 47개
04. c2_tat 이상치 제거 : 1개
05. dc2_tat 이상치 제거 : 1개
06. c3_tat 이상치 제거 : 0개 (이전 단계에서 삭제됨)
07. dc3_tat 이상치 제거 : 0개 (이전 단계에서 삭제됨)
08. c4_tat 이상치 제거 : 0개 (이전 단계에서 삭제됨)
09. pg1_tat 이상치 제거 : 1개
10. sa1_tat 이상치 제거 : 0개 (이전 단계에서 삭제됨)
11. sa2_tat 이상치 제거 : 7개
12. sa3_tat 이상치 제거 : 2개

total 제거 : 121개 (전체 6%)
"""
#%% error_rate

"""
tray엔 총 25cell이 있어야 하는 것을 생각하여, 25이 안되는 경우 어떻게 해야하나? 논란
-> 교수님 피드백 : 꼭 tray에 25개가 있을 필요는 없다고 판단하여

총 불량 개수/총 tray당 cell 개수를 count해 불량률 생성
다만 불량률 차이를 위해 0.01정도 차이나는 cell count 20 미만 제거
"""

tray_list = pd.merge( df_tat[["lot_id",'tray_id']], pd.DataFrame( df_raw_process.groupby(["lot_id","tray_id"])['cell_id'].count()).reset_index(drop = False) )
tray_list = pd.merge( tray_list, df_raw_process[ df_raw_process["judge"] == "불량" ].groupby(["lot_id","tray_id"])['cell_id'].count().reset_index(drop = False) ,
                     on = ["lot_id","tray_id"], how = "outer")
tray_list.fillna(0, inplace = True)

tray_list["error_rate"] = tray_list[tray_list.columns[3]] / tray_list[tray_list.columns[2]] # 브로드 캐스팅
tray_list_cut = tray_list[ tray_list["cell_id_x"] >= 20 ]


df_tat = pd.merge( df_tat, tray_list_cut[["lot_id","tray_id", "error_rate"]], on = ["lot_id","tray_id"], how = "inner")
df_tat = df_tat[ df_raw_tat.columns.insert(2, "error_rate").to_list() ] # y = 불량률(파생변수)

rta2_tat 이상치 제거 전 :  1797
rta2_tat 이상치 제거 후 :  1736 

ocv1_tat 이상치 제거 전 :  1736
ocv1_tat 이상치 제거 후 :  1735 

c1_tat 이상치 제거 전 :  1735
c1_tat 이상치 제거 후 :  1688 

c2_tat 이상치 제거 전 :  1688
c2_tat 이상치 제거 후 :  1687 

dc2_tat 이상치 제거 전 :  1687
dc2_tat 이상치 제거 후 :  1686 

c3_tat 이상치 제거 전 :  1686
c3_tat 이상치 제거 후 :  1686 

dc3_tat 이상치 제거 전 :  1686
dc3_tat 이상치 제거 후 :  1686 

c4_tat 이상치 제거 전 :  1686
c4_tat 이상치 제거 후 :  1686 

pg1_tat 이상치 제거 전 :  1686
pg1_tat 이상치 제거 후 :  1685 

sa1_tat 이상치 제거 전 :  1685
sa1_tat 이상치 제거 후 :  1685 

sa2_tat 이상치 제거 전 :  1685
sa2_tat 이상치 제거 후 :  1678 

sa3_tat 이상치 제거 전 :  1678
sa3_tat 이상치 제거 후 :  1676 



# 카이제곱검정

# Aging 단계 상온1

In [3]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['rta1_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 9.5268
P-value: 0.217


In [4]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['rta1_box_row'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 659.0818
P-value: 0.0


In [5]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['rta1_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 74.4553
P-value: 0.0


# 고온 aging 단계1

In [6]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['hta1_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 9.2857
P-value: 0.0096


In [7]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['hta1_box_row'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 486.7929
P-value: 0.0


In [8]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['hta1_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 75.1714
P-value: 0.0


# 상온 aging 단계2

In [9]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['rta2_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 59.1017
P-value: 0.0


In [10]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['rta2_box_row'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 672.5357
P-value: 0.0


In [11]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['rta2_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 71.66
P-value: 0.0


# charge1단계

In [78]:
df_process['judge']

0         
1         
2         
3         
4         
        ..
38590     
38591     
38592     
38593     
38594     
Name: judge, Length: 38595, dtype: object

In [77]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
cross = pd.crosstab(index = df_process['c1_box_col'], columns = df_process['judge'])
#contingency_table = pd.crosstab()

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(cross)

print("Chi-Squared Value:", chi2)
print("P-value:", p)

Chi-Squared Value: 0.0
P-value: 1.0


In [74]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2)
print("P-value:", p)

Chi-Squared Value: 0.0
P-value: 1.0


In [76]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_dan' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2)
print("P-value:", p)

Chi-Squared Value: 0.0
P-value: 1.0


In [15]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_curr_end'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 52.8108
P-value: 0.0


In [16]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_voltage_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 122.7481
P-value: 0.0


In [17]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_capa'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 720.9624
P-value: 0.0


In [18]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_time_cc'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 617.7249
P-value: 0.0


In [19]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_ccval'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 720.9624
P-value: 0.0


In [20]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c1_temp_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3357.1693
P-value: 0.0


# Discharge 1 단계

In [21]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc1_curr_end'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 105.8148
P-value: 0.0


In [22]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc1_capa'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 0.4969
P-value: 0.4809


In [23]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_ccval'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 279.8424
P-value: 0.0


In [24]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc1_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 682.396
P-value: 0.0


In [25]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc1_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 241.3546
P-value: 0.0


In [26]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc1_temp_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3477.9231
P-value: 0.0


In [27]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc1_capafit'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3134.8199
P-value: 0.0


# charge2단계

In [28]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 682.396
P-value: 0.0


In [29]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 241.3546
P-value: 0.0


In [30]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_curr_end'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 1272.3151
P-value: 0.0


In [31]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_voltage_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 222.4093
P-value: 0.0


In [32]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_capa'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 279.8424
P-value: 0.0


In [33]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_ccval'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 279.8424
P-value: 0.0


In [34]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_time_cc'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 216.7892
P-value: 0.0


In [35]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c2_temp_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3627.2479
P-value: 0.0


# discharge2단계

In [36]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc2_curr_end'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 113.8512
P-value: 0.0


In [37]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc2_voltage_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 1555.3603
P-value: 0.0


In [38]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc2_capa'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 0.1038
P-value: 0.7473


In [39]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc2_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 682.396
P-value: 0.0


In [40]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc2_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 241.3546
P-value: 0.0


In [41]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc2_temp_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3344.1658
P-value: 0.0


In [42]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc2_capafit'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 2623.7876
P-value: 0.0


# charge3 단계

In [43]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 682.396
P-value: 0.0


In [44]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 241.3546
P-value: 0.0


In [45]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_curr_end'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 1.5068
P-value: 0.2196


In [46]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_ccval'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 160.7349
P-value: 0.0


In [47]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_capa'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 1053.024
P-value: 0.0


In [48]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_voltage_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 306.0135
P-value: 0.0


In [49]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_time_cc'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 139.3361
P-value: 0.0


In [50]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c3_temp_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3346.8962
P-value: 0.0


# discharge 3단계

In [51]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc3_curr_end'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 19.4196
P-value: 0.2475


In [52]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc3_voltage_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 363.7364
P-value: 0.0


In [53]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc3_capa'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 458.7546
P-value: 0.0


In [54]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc3_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 682.396
P-value: 0.0


In [55]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc3_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 241.3546
P-value: 0.0


In [56]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc3_temp_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3103.1731
P-value: 0.0


In [57]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['dc3_capafit'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 307.4222
P-value: 0.0


# charge 4단계

In [58]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_box_col'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 682.396
P-value: 0.0


In [59]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_box_dan'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2)
print("P-value:", p)

Chi-Squared Value: 241.35455107910906
P-value: 3.559014190487565e-46


In [60]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_curr_end'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 0.0008
P-value: 0.9774


In [61]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_voltage_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 365.3459
P-value: 0.0


In [62]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_capa'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 193.5058
P-value: 0.1738


In [63]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_ccval'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 612.6281
P-value: 0.0


In [64]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_time_cv'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 1064.3583
P-value: 0.0


In [65]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_cvval'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 976.5527
P-value: 0.0


In [66]:
import pandas as pd
from scipy.stats import chi2_contingency

#'c1_box_col' 변수와 'judge' 변수 간의 교차 테이블 생성
contingency_table = pd.crosstab(df_process['c4_temp_avg'], df_process['judge'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Squared Value:", chi2.round(4))
print("P-value:", p.round(4))

Chi-Squared Value: 3273.637
P-value: 0.0


# DT

In [67]:
import pandas as pd
import seaborn as sns

#%% 데이터 불러오기
df_tat = pd.read_csv("bat_tat.csv")
df_process = pd.read_csv("bat_process.csv", encoding = 'cp949')

#%% 불량률
tray_list = pd.merge( df_tat[['tray_id']], pd.DataFrame( df_process.groupby("tray_id")['cell_id'].count()).reset_index(drop = False) )
tray_list = pd.merge( tray_list, df_process[ df_process["judge"] == "불량" ].groupby("tray_id")['cell_id'].count().reset_index(drop = False) ,
                     on = "tray_id", how = "outer")
tray_list.fillna(0, inplace = True)
tray_list["odds_ratio"] = tray_list[tray_list.columns[2]] / tray_list[tray_list.columns[1]] # 브로드 캐스팅
#%%

"""
내가 접근을 잘못했나 불량률 어떻게 하는거지...
내가 뭔가 잘못하고 있는거 같은데 내가 뭘 잘못하고 있는지 잘 모르겠다
"""
print(tray_list["odds_ratio"].mean()) # 불량률이 어떻게 될까
#%%

0.031244500878199053


In [68]:
tray_list = pd.merge( df_tat[["lot_id",'tray_id']], pd.DataFrame( df_process.groupby(["lot_id","tray_id"])['cell_id'].count()).reset_index(drop = False) )
tray_list = pd.merge( tray_list, df_process[df_process["judge"] == "불량" ].groupby(["lot_id","tray_id"])['cell_id'].count().reset_index(drop = False) ,
                     on = ["lot_id","tray_id"], how = "inner")

tray_list.fillna(0, inplace = True)
tray_list["odds_ratio"] = tray_list[tray_list.columns[3]] / tray_list[tray_list.columns[2]] # 브로드 캐스팅
tray_list_cut = tray_list[ tray_list["cell_id_x"] >= 20 ]
df_tat = pd.merge( df_tat, tray_list_cut[["lot_id","tray_id", "odds_ratio"]], on = ["lot_id","tray_id"], how = "inner")
df_tat = df_tat[ df_tat.columns.insert(2, "odds_ratio").to_list() ] # y = 불량률(파생변수)

# 한글 제거
for column in df_process.columns:
    if df_process[column].dtype == 'object' and df_process[column].str.contains('[가-힣]').any():
        df_process[column] = df_process[column].str.replace('[가-힣]', '', regex=True)

# 설비간 불량률 C구역 데이터프레임 생성
df_rcd = df_process.select_dtypes(include="object")
df_rcd.drop_duplicates(['lot_id', 'tray_id'], keep='first', inplace=True)

# 'c1_box_col'과 'c1_box_dan' 컬럼에서 한글이 제거되었으므로, 이제 정수 변환을 시도해 볼 수 있습니다.
df_rcd['c1_box_col'] = df_rcd['c1_box_col'].str.extract('(\d+)').astype(int)
df_rcd['c1_box_dan'] = df_rcd['c1_box_dan'].str.extract('(\d+)').astype(int)


In [69]:
df_rcd

,lot_id,tray_id,cell_id,judge,dt_start,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,...,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,sa2_box_col,sa2_box_row,sa2_box_dan,sa3_box_col,sa3_box_row,sa3_box_dan
0,LOT-100,TRAY-1002,CELL-10300,,25NOV2023:05:17:55,04,08,04,03,20,...,01,16,09,04,11,31,04,16,54,10
25,LOT-100,TRAY-1004,CELL-10003,,25NOV2023:05:45:20,07,40,01,02,03,...,01,10,17,02,10,63,06,06,59,08
50,LOT-100,TRAY-1005,CELL-10001,,25NOV2023:06:00:55,01,13,01,01,02,...,01,01,64,08,06,59,08,09,50,08
75,LOT-100,TRAY-1007,CELL-10076,,25NOV2023:06:19:50,07,05,01,01,24,...,01,08,37,12,14,44,02,10,48,05
100,LOT-100,TRAY-1008,CELL-10175,,25NOV2023:06:36:25,07,05,01,01,24,...,01,08,37,12,14,44,02,10,48,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38474,LOT-144,TRAY-2786,CELL-53783,,11DEC2023:17:33:20,07,17,01,01,23,...,01,11,54,08,14,05,08,11,16,13
38497,LOT-144,TRAY-2788,CELL-53506,,11DEC2023:17:59:00,03,08,02,02,23,...,01,16,08,04,16,55,11,13,29,03
38521,LOT-144,TRAY-2789,CELL-53508,,11DEC2023:18:14:50,03,08,02,02,23,...,01,15,39,13,10,23,09,01,60,06
38546,LOT-144,TRAY-2790,CELL-53654,,11DEC2023:18:23:55,08,21,03,02,23,...,01,05,43,09,07,04,05,04,55,13


In [70]:
# df_rcd_c 데이터프레임의 'c1_box_dan'과 'c1_box_col' 컬럼 값 확인
print(df_rcd_c[['c1_box_dan', 'c1_box_col']].drop_duplicates())

# 필터링 조건에 맞는 데이터가 있는지 확인
filtered_data = df_rcd_c.loc[df_rcd_c["c1_box_dan"].isin([1, 2, 3, 4]) & df_rcd_c["c1_box_col"].isin([22, 21, 23])]
print("필터링된 데이터 수:", len(filtered_data))

NameError: name 'df_rcd_c' is not defined

# df_sel => C그룹

In [ ]:
df_process["c1_box_dan"] = df_process["c1_box_dan"].astype(int)
df_process["c1_box_col"] = df_process["c1_box_col"].astype(int)

In [ ]:
df_sel = df_process.query("c1_box_dan <= 4 and c1_box_col >= 20")

In [ ]:
df_sel['judge']

In [ ]:
# #%% 설비간 불량률 C구역
# df_rcd = df_process.select_dtypes(include = "object")
# df_rcd.drop_duplicates(['lot_id', "tray_id"], keep="first", inplace = True)
# df_rcd_c = df_rcd[["lot_id", "tray_id", "c1_box_col", "c1_box_dan"]]
# df_rcd_c = pd.merge( tray_list_cut , df_rcd_c, on = ["lot_id", "tray_id"], how = "inner")
# df_rcd_bad = df_rcd_c.loc[ df_rcd_c["c1_box_dan"].isin([1, 2, 3, 4])]
# df_rcd_bad = df_rcd_bad.loc[ df_rcd_bad["c1_box_col"].isin([22, 21, 23])]

# # #%% A열
# # df_rcd_good = df_rcd_c.loc[ df_rcd_c["c1_box_dan"].isin(["08단", "09단", "10단", "11단"])]
# # df_rcd_good = df_rcd_good.loc[ df_rcd_good["c1_box_col"].isin(["14열", "13열", "12열"])]
# # print(df_rcd_good["odds_ratio"].mean())

# # #%% B 구역
# # df_rcd_b = df_rcd_c.loc[ df_rcd_c["c1_box_dan"].isin(["04단", "05단", "06단", "07단"])]
# # df_rcd_b = df_rcd_b.loc[ df_rcd_b["c1_box_col"].isin(["10열", "12열", "11열"])]
# # print(df_rcd_b["odds_ratio"].mean())

In [ ]:
# df_rcd_bad

## C구역과 B구역만 의사결정나무 돌리기

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 타겟 변수('judge')와 특성 변수 분리
X = df_sel.drop(columns=['judge'])  # 'judge' 컬럼을 제외한 모든 컬럼을 특성 변수로 사용
y = df_sel['judge']  # 'judge' 컬럼을 타겟 변수로 사용

# 데이터 분할: train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# 의사결정트리 분류 모델 생성 및 훈련
decision_tree = DecisionTreeClassifier(random_state=1234)
decision_tree.fit(X_train, y_train)

In [ ]:
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# # C 구역 데이터 필터링
# df_rcd_bad = df_rcd_c.loc[df_rcd_c["c1_box_dan"].isin(["01단", "02단", "03단", "04단"]) & df_rcd_c["c1_box_col"].isin(["22열", "21열", "20열"])]

# # 데이터 분할: X(특성)와 y(타겟)
# X = df_rcd_bad.drop(columns=['odds_ratio', 'lot_id', 'tray_id', 'c1_box_col', 'c1_box_dan'])  # 'odds_ratio'는 타겟 변수이므로 제외, 다른 범주형 컬럼도 제외
# y = df_rcd_bad['odds_ratio']  # 'odds_ratio'를 타겟 변수로 사용

# # 데이터 분할: train, test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# print("분할 전 설명변수 데이터 :", X.shape)
# print("분할 후 설명변수 데이터 :Train", X_train.shape, "  Test",X_test.shape)

# # # 의사결정트리 모델 생성 및 훈련
# # dt_reg = DecisionTreeRegressor(random_state=42)
# # dt_reg.fit(X_train, y_train)

# # # 모델 성능 평가
# # y_pred = dt_reg.predict(X_test)
# # mse = mean_squared_error(y_test, y_pred)
# # print("MSE:", mse)

df_rcd_c에서 C 구역의 데이터를 필터링하고, 이 데이터를 사용하여 odds_ratio에 대한 의사결정트리 모델을 훈련하고 평가. X에 사용된 특성 데이터는 수정이 필요할 수 있으며, 실제 사용 가능한 수치형 특성을 기반으로 선택해야 합니다. 또한, 범주형 변수를 포함시키고 싶은 경우, 적절한 인코딩 방법(예: 원-핫 인코딩)을 적용

## Default option Model

In [ ]:
# 의사 결정 나무 모델 초기화 및 훈련
decision_tree = DecisionTreeClassifier(random_state=1234)
decision_tree.fit(X_train, y_train)

# train 데이터 정확도
print("Accuracy on training set: {:.3f}".format(decision_tree.score(X_train, y_train)))
# test 데이터 정확도
print("Accuracy on test set: {:.3f}".format(decision_tree.score(X_test, y_test)))